In [1]:
import wikipedia
import pandas as pd 
import numpy as np

In [2]:
wikipedia.set_lang("en")
statistics = wikipedia.search("statistics")

In [3]:
statistics = wikipedia.page("Statistics")
#print(statistics.title)
#print(statistics.url)
#print(statistics.content)
#print(statistics.links[0])


In [4]:
wikipedia.summary("Statistics", sentences=1)

'Statistics is a branch of mathematics working with data collection, organization, analysis, interpretation and presentation.'

In [5]:
#! pip3 install wikipedia-api


In [6]:
#! pip install wiki-dump-reader


In [7]:
from wiki_dump_reader import Cleaner, iterate

corpus = []
cleaner = Cleaner()
for title, text in iterate('data/enwiki-latest-pages-articles.xml'):
    text = cleaner.clean_text(text)
    cleaned_text, links = cleaner.build_links(text)
    corpus.append(cleaned_text)


In [8]:
type(corpus)

list

In [9]:
len(corpus)

29316

In [10]:
len(corpus[46])

5199

In [11]:
long_text = []
for i in corpus:
    if len(i) > 100:
        long_text.append(i)
    

In [12]:
len(long_text)

10102

In [14]:
sample_text = long_text[0:10]

In [126]:
from nltk import PorterStemmer
PorterStemmer().stem('complications')

'complic'

In [120]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sherzyang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [121]:
docs = long_text

In [128]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [129]:
words = []
for word in docs[0].split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(docs[0]))

['Louis', 'J.', 'Russell', '(circa', '1912–1973)', 'was', 'an', 'agent', 'of', 'the', 'Federal', 'Bureau', 'of', 'Investigation', '(FBI),', 'senior', 'investigator', 'of', 'the', 'House', 'Un-American', 'Activities', 'Committee', '(HUAC),', 'and', 'private', 'detective.\n==Background==\nLouis', 'James', 'Russell', 'graduated', 'from', 'the', 'Catholic', 'University', 'of', 'America.\n==Career==\nIn', '1935,', 'Russell', 'joined', 'the', 'FBI', 'as', 'an', 'agent.\n===HUAC===\nIn', '1945,', 'Russell', 'joined', 'HUAC', 'as', 'an', 'investigator.', '', 'Robert', 'E.', 'Stripling', 'has', 'Russell', 'testify', 'on', 'what', 'he', 'knew', 'about', 'Gerhart', 'Eisler', 'and', 'Hollywood', 'industry', 'people.', '', 'He', 'also', 'testified', 'about', 'Leon', 'Josephson', 'and', 'Alexander', 'Koral.', '', 'By', '1948,', 'Russell', 'was', 'a', 'HUAC', 'senior', 'investigator', 'in', 'the', 'Hiss-Chambers', 'case.', '', 'In', 'his', 'memoir', 'Six', 'Crises,', 'Richard', 'Nixon', 'recalled', '

In [133]:
processed_docs = list(map(lambda x: preprocess(x), docs))
#processed_docs[:10]

In [134]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break


0 accord
1 activities
2 actor
3 agency
4 agenda
5 agent
6 agents
7 aide
8 alch
9 alcoholic
10 alexander


In [135]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


In [165]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [149]:
df_words = pd.DataFrame()

In [157]:
words = []
count = []
for i in bow_corpus:
    for text in range(len(i)):
        temp = "{}".format(dictionary[i[text][0]])
        appearance = "{}".format(i[text][1])
        words.append(temp)
        count.append(appearance)
        

In [158]:
df_words['count'] = pd.Series(count)
df_words['word'] = pd.Series(words)

In [164]:
df_words = df_words.drop(['word number'], axis=1)

In [168]:
df_words = df_words[['word','count']]
df_words.sort_values(by=['count'])

,word,count
0,accord,1
6214,century,1
6213,birth,1
6212,write,1
6211,serve,1
6209,return,1
6207,originally,1
6205,investigate,1
6204,guard,1
6203,die,1


In [193]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint

vectors = []
for doc in corpus_tfidf:
    vectors.append(doc)

In [203]:
flat_list = [item for sublist in vectors for item in sublist]
vec = pd.DataFrame(flat_list)

In [209]:
vec['tfidf_score'] = vec[1]

In [210]:
vec['word'] = vec[0]

In [212]:
vec = vec.drop([0,1],axis=1)

In [225]:
four = []
three = []
two = []
one = []
for row in vec['tfidf_score']:
    if row > 0.110191: 
        four.append(row)
    elif 0.110191 < value <= 0.059553:
        three.append(row)
    elif 0.032652 < value <= 0.059553:
        two.append(row)
    elif value <= 0.032652:
        one.append(row)
        

In [226]:
print("len_one:" + str(len(one)) + " , len_two:" + str(len(two)) 
      + " , len_three:" + str(len(three)) + " , len_four:" + str(len(four)))

len_one:407533 , len_two:0 , len_three:0 , len_four:135844


# Previous model 

In [169]:
import pandas as pd
 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [170]:
docs = long_text

In [171]:
#instantiate CountVectorizer()
cv=CountVectorizer()
 
# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(docs)

In [172]:
word_count_vector.shape


(10102, 114203)

In [173]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [174]:
# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["tf_idf_weights"])

In [175]:
# sort ascending
df_idf_sorted = df_idf.sort_values(by=['tf_idf_weights'], ascending=False)
     

In [180]:
# count matrix
count_vector=cv.transform(docs)
 
# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [181]:
feature_names = cv.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]
 
#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)
 

,tfidf
russell,0.674132
huac,0.300696
watergate,0.215924
mccord,0.174262
hiss,0.139410
fensterwald,0.112761
to,0.111421
investigator,0.105638
security,0.105441
nixon,0.092585


In [191]:
df.describe()

,tfidf
count,114203.000000
mean,0.000090
std,0.002958
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,0.674132


In [182]:
words_difficulty = df.to_dict()

In [183]:
words_difficulty = list(words_difficulty.values())

In [184]:
words_difficulty = words_difficulty[0]

In [185]:
#four = {k:v for k,v in words_difficulty.items() if v > 2}

In [188]:
four = []
three = []
two = []
one = []
for i in words_difficulty.items():
    if value > 0.110191: 
        four.append((key,value))
    elif 0.110191 >= value > 0.059553:
        three.append((key,value))
    elif 0.059553 >= value > 0.032652:
        two.append((key,value))
    elif value <= 0.032652:
        one.append((key,value))    

In [189]:
print("len_one:" + str(len(one)) + " , len_two:" + str(len(two)) 
      + " , len_three:" + str(len(three)) + " , len_four:" + str(len(four)))

len_one:114201 , len_two:1 , len_three:1 , len_four:0


In [ ]:
def split_text(text):
    """ 
    Takes in a corpus and counts how many words are part of a list of words in the 75th+ percentile of sample_text
    , counts how many words are part of a list of words in the 50-75th+ percentile of sample_text
    , counts how many words are part of a list of words in the 25-50th+ percentile of sample_text
    , counts how many words are part of a list of words in the 0-25th percentile of sample_text
    """
    for i in ___: 
        if 